# A Regression Playground

EECS 127/227a

In this notebook, we will explore 4 different types of regression: Ordinary Least Squares, Ridge Regression, Weighted Least Squares, and Tikhonov Regularization. Only modify code in the sections marked TODO and fill in your answers to the questions marked TODO.

## Setup - Imports & Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from matplotlib.font_manager import FontProperties
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
np.set_printoptions(3, suppress=True)
fontP = FontProperties()
fontP.set_size('small')

In [ ]:
np.random.seed(42)

In [ ]:
def data_fn(X):

    y = 1/1000*(X - 6)*(X - 4)*(X - 2)*(X + 1)*(X + 3)*(X + 5)*(X + 7)
    
    return y

In [ ]:
def generate_data(m, n, poly_degree=1, generate_bad_data=False):

    X = np.linspace(-7, 6, m).reshape(-1, 1)
    
    y = data_fn(X)
    
    noise = np.random.normal(0, 2, size=X.shape[0]).reshape(-1, 1)
    y_obs = y + noise
    
    bad_noise = np.random.normal(0, 10, size=X.shape[0]).reshape(-1, 1)
    y_obs_bad = y + bad_noise
    
    # create polynomial features
    poly = PolynomialFeatures(degree=poly_degree)
    X_hat = poly.fit_transform(X)

    X_true = np.linspace(-7.1, 6.1, 1000).reshape(-1, 1)
    y_true = data_fn(X_true)
    
    X_hat_true = poly.fit_transform(X_true)
    
    # Note we are creating "test" data here, but this is really like a true error, since we have the underlying polynomial at hand
    # So even though we add some noise, this is a bit artificial
    y_test = data_fn(X) + np.random.normal(0, 2, size=X.shape[0]).reshape(-1, 1)
    
    return X_hat, X, X_hat_true, X_true, y_obs, y_obs_bad, y_true, y_test

In [ ]:
def plot_result(X_hat, X, X_true, y_obs, y_true, y_test, ws, y_obs_bad=None):
    
    fig, ax = plt.subplots(1, 2, figsize=(12,4))
    
    ax[0].set_title("Regression")
    ax[0].set_xlabel("x")
    ax[0].set_ylabel("y")
    ax[0].grid()
    ax[0].scatter(X, y_obs, label="Observations", alpha=0.5, color="green", marker="+")
    ax[0].plot(X_true, y_true, label="Groud Truth", alpha=0.5, color="orange", linestyle="--")
    ax[0].scatter(X, y_test, label="Test Set", color="blue", alpha=0.5, marker="*")
    
    if y_obs_bad is not None:
        ax[0].scatter(X, y_obs_bad, label="Bad Observations", alpha=0.5, color="red", marker="x")
    
    for w, name in ws:
        ax[0].scatter(X, X_hat @ w, label=name)
    ax[0].legend(prop=fontP)

    ax[1].set_title("Visualizing w")
    ax[1].grid()
    
    for w, name in ws:
        ax[1].scatter(w[0][0], w[1][0], label=name)
    ax[1].legend()

## Setup - Data Generation
Our data will be generated from some pre-defined function, but with some Gaussian/random noise added to it. Our goal will be to fit a 7-degree polynomial to this data. We have two sets of observations. "Good observations" are those which were collected with a small amount of noise, but "Bad observations" were collected with much more noise.

In [ ]:
X_hat, X, X_hat_true, X_true, y_obs, y_obs_bad, y_true, y_test = generate_data(30, 1, poly_degree=7)

In [ ]:
plt.title("Data Generated")
plt.xlabel("x")
plt.ylabel("y")
plt.plot(X_true, y_true, label="Ground Truth", color="orange")
plt.scatter(X, y_obs, label="Good Observations", color="green", marker="+")
plt.scatter(X, y_obs_bad, label="Bad Observations", color="red", marker="x")
plt.scatter(X, y_test, label="Test Set", color="blue", marker="*")
plt.legend(prop=fontP)
plt.grid()

##  Ordinary Least Squares (OLS)
We'll begin with the standard least-squares problem. Given some data in the form of $(x, y)$ pairs, we wish to find the polynomial that will minimize the sum of the squared errors (SSE):

$$\min \sum_{i}(y_i - w^\top x_i)^2 = \min_{w} \|Xw - y\|_2$$

Note that even though we are fitting a polynomial, the objective is still linear in the weights vector $w$. This is because we create   features in the data matrix corresponding to higher order polynomial terms:

$$X = \begin{bmatrix}x_1 \\ x_2 \\ \vdots \\ x_m \end{bmatrix} \rightarrow \hat{X} = \begin{bmatrix} x_1 & x_1^2 & \dots & x_1^d \\ x_2 & x_2^2 & \dots & x_2^d \\ \vdots \\ x_m & x_m^2 & \dots & x_m^d \end{bmatrix}$$

So now, each $w_i$ tunes the effect of adding an $i$th order term. This is known as $\textbf{lifting}$ the data into higher-dimensional space. Finding the "line" that regresses in this space is equivalent to finding the higher dimensional polynomial that actually fits the data.

Disclaimer: For this example, we will ignore the bad observations. We will return to that problem later.

### a). TODO: Implement OLS

In [ ]:
def OLS(X, y, verbose=False):
    
    # TODO: Implement OLS 
    # Fill in the expressions for the pseudoinverse, X_dagger, and use that to compute the optimal w
    
    #----------
    X_dagger = # complete this
    w = # complete this to find optimal w 
    
    #----------
    error = np.linalg.norm(X @ w - y)
    
    if verbose:
        print("--------------------OLS-----------------------\n")
        print("The optimal w is", np.ndarray.flatten(w))
        print("\n")
        print("Training error:", error)
        print("\n")
    
    return w, error

In [ ]:
w_ls, train_error_ls = OLS(X_hat, y_obs, verbose=True)
test_error_ls = np.linalg.norm(X_hat @ w_ls - y_test)
print("Testing error:", test_error_ls)

In [ ]:
# Note that the 7 dimensional w is projected into two dimensions for plotting.
plot_result(X_hat, X, X_true, y_obs, y_true, y_test, [[w_ls, "OLS"]])

# Ridge $(\ell_2)$ Regression
Vanilla least squares is great, but can be affected negatively by noise. Ordinary Least Squares (OLS) is susceptible to overfitting the data, since we only care about minimizing the squared error of the data we trained it on.

We can attempt to prevent this overfitting by introducing a penalty term on the weight vector. If we penalize it from getting too large (numerically unstable), we can combat the impact of noise. The penalty term we will use here is the $\ell_2$-norm:

$$\min_{w} \|Xw - y\|_2^2 + \lambda\|w\|_2^2$$

### b.ii) TODO: Implement Ridge Regression 

In [ ]:
ridge_lmbda = None
ridge_lambdas = np.linspace(.0001,20, 10000)


def ridge(X, y, lmbda, verbose=False):
    
    # TODO: Implement Ridge Regression
    
    #----------
    
    X_dagger = # Complete this
    w = # Complete this to find optimal w
    #----------
    error = np.linalg.norm(X @ w - y)
    
    if verbose:
        print("--------------------Ridge-----------------------\n")
        print("Lambda: ", lmbda)
        print("\n")
        print("The optimal w is", np.ndarray.flatten(w))
        print("\n")
        print("Training error:", error)
        print("\n")
        
    return w, error

## Hyperparmater Tuning
One thing to note is the $\lambda$ in the problem. It is known as a $\textbf{hyperparameter}$ and must be manually tuned, since we don't really know how much to penalize the norm. In order to find the best $\lambda$ we test different values and see how well the learned model generalizes by computing the squared  error of the predictions of our model on unseen test data. Below, you can see the effect of different values of $\lambda$ on the SSE for the training data, the SSE for the test data, and the components of $w$.

In [ ]:
train_errors_rr = []
test_errors_rr = []
ws_rr = []
for l in ridge_lambdas:
    w_rr, train_error_rr = ridge(X_hat, y_obs, l)
    test_error_rr = np.linalg.norm(X_hat @ w_rr - y_test)
    train_errors_rr.append(train_error_rr)
    ws_rr.append(w_rr)
    test_errors_rr.append(test_error_rr)

In [ ]:
fig = plt.figure(figsize=(12,4))

fig.add_subplot(121)
plt.title("Error")
plt.xlabel(r"$\lambda$")
#plt.xscale("log")
plt.plot(ridge_lambdas, train_errors_rr, label="Training Error")
plt.plot(ridge_lambdas, test_errors_rr, label="Test Error")
plt.legend()

fig.add_subplot(122)
plt.title("Regularization Path of w")
num_coeffs = len(ws_rr[0])
plt.xlabel(r"$\lambda$")
#plt.xscale("log")
for i in range(num_coeffs):
    plt.plot(ridge_lambdas, [w[i][0] for w in ws_rr], label="w_" + str(i))
plt.legend(prop=fontP)

###  Reflect on the above graphs: What do you observe in the graphs above? Why does increasing $\lambda$ lead to these trends in the training error, testing error, and weights?

In [ ]:
# TODO: Try changing these values, what do you see?
test_lmbdas = [.1, 1, 2, 100] 

In [ ]:
w_rrs = []
for l in test_lmbdas:
    w_rr, train_error_rr = ridge(X_hat, y_obs, l, verbose=True)
    test_error_rr = np.linalg.norm(X_hat @ w_rr - y_test)
    print("Testing error:", test_error_rr)
    print("\n")
    w_rrs.append([w_rr, r"$\lambda:$" + str(l)])
w_rrs.append([w_ls, "OLS"])

In [ ]:
plot_result(X_hat, X, X_true, y_obs, y_true, y_test, w_rrs)

# Weighted Least Squares (WLS)
Sometimes, our data observations $X$ may not be trustworthy. Perhaps multiple sources gathered data, but one is very noisy. How can we account for this? We can just weight each of the data points! Ideally, we would like data points that come from "trustworthy" sources to have a greater weight, so we can effectively perform a change of basis to a space that reflects this.
$$\min_{w} \|W_1(Xw - y)\|_2$$
$$W_1 = diag(w_1, w_2, \dots)$$

To show the usefulness, we can compare how OLS would react to this extra "bad" data versus WLS.

In [ ]:
# append bad data to data matrix
X_hat_with_bad = np.vstack((X_hat, X_hat))
y_with_bad = np.vstack((y_obs, y_obs_bad))

### c.i) TODO: Implement WLS 

In [ ]:
def WLS(X, y, W_vec, verbose=False):
    
    W = np.diag(W_vec)
    
    # TODO: Implement WLS using the pseudoinverse
    #----------
    

    X_dagger = # complete this
    w = # complete this to find optimal w
    
    #----------
    error = np.linalg.norm(X @ w - y)
    
    if verbose:
        print("--------------------WLS-----------------------")
        print("\n")
        print("The optimal w is", np.ndarray.flatten(w))
        print("\n")
        print("Training error:", error)
        print("\n")
    
    return w, error

In [ ]:
# Choose appropriate data weight vector (this will eventually become a diagonal matrix)
# We only trust the first half of the data more than the second half

wls_W1_vec = np.concatenate((np.ones(X.shape[0]), 0.5*np.ones(X.shape[0])))

In [ ]:
w_ls_bad, train_error_ls_bad = OLS(X_hat_with_bad, y_with_bad, verbose=True)
test_error_ls_bad = np.linalg.norm(X_hat @ w_ls_bad - y_test)
print("Testing error:", test_error_ls_bad)
print("\n")

w_wls, train_error_wls = WLS(X_hat_with_bad, y_with_bad, wls_W1_vec, verbose=True)
test_error_wls = np.linalg.norm(X_hat @ w_wls - y_test)
print("Testing error:", test_error_wls)
print("\n")

w_wls = [[w_wls, "WLS"], [w_ls_bad, "OLS w/ Bad Data"]]

In [ ]:
plot_result(X_hat, X, X_true, y_obs, y_true, y_test, w_wls, y_obs_bad=y_obs_bad)

### c.ii) TODO: Comment on the difference between WLS and OLS w/ Bad Data, which one performed better and why?

## (optional) WLS Playground

We used a very straightforward approach in how we treated the validity of the data. But it didn't turn out to work that well! The training error of OLS was actually pretty good compared to WLS, so perhaps stricly ignoring the data isn't the best choice. Now it's your turn to play around with different $W_1$ values. See what happens when you choose different ones!

In [ ]:
# Choose appropriate data weight vector (this will eventually become a diagonal matrix)

# TODO: play around with this vector and see the results
#----------

wls_W1_vec = np.concatenate((np.ones(X.shape[0]), np.zeros(X.shape[0])))

#----------

w_wls, train_error_wls = WLS(X_hat_with_bad, y_with_bad, wls_W1_vec, verbose=True)
test_error_wls = np.linalg.norm(X_hat @ w_wls - y_test)
print("Testing error:", test_error_wls)
print("\n")

w_wls = [[w_wls, "WLS"]]

plot_result(X_hat, X, X_true, y_obs, y_true, y_test, w_wls, y_obs_bad=y_obs_bad)

### TODO (optional): What $W_1$s did you try? How did they affect the regressor?

# Tikhonov Regularization

Tikhonov regularization is the combination of everything we've seen so far: ordinary least squares (OLS), giving weight to each sample via the $W_1$ matrix, and $\ell_2$-norm penalization. However, we can also generalize the last penalty: instead of one $\lambda$, use another another matrix $W_2$ that gives penalization weights to each element. For example, if we really cared about only allowing low-order polynomial terms, we would $\textit{decrease}$ the weights on the first few $w_i$s, as we don't wish to penalize "using" those. Putting this all together, we get the following form:

$$\min_{w} \|W_1(Xw - y)\|_2^2+ \|W_2(w - w_0)\|_2^2$$
$$W_1 = diag(w_1, w_2, \dots), W_2 = diag(\lambda_1, \lambda_2, \dots)$$

One thing that may seem new is the $w - w_0$ term. To understand this, consider what we saw before: we just penalized the vector $w$, meaning that the longer $w$ was from the origin, the higher the penalty. However, we can generalize this as well: instead, we can penalize how far it is from some predetermined "center" $w_0$.

Since this is the most robust method, we will juxtapose it with the performance of all methods seen so far on the entire good + bad dataset.

### d) Implement Tikhonov Regularization 

In [ ]:
def tikhonov(X, y, W1_vec, W2_vec, w_0, verbose=False):
    
    W1 = np.diag(W1_vec)
    W2 = np.diag(W2_vec)
    #----------
    
    
    
    X_tilde = # Complete this
    w = # Complete this to find optimal w
    
    #----------
    error = np.linalg.norm(X @ w - y)
    
    if verbose:
        print("--------------------Tikhonov-----------------------\n")
        print("The optimal w is", np.ndarray.flatten(w))
        print("\n")
        print("Training error:", error)
        print("\n")

    return w, error

In [ ]:
w_ls_bad, train_error_ls_bad = OLS(X_hat_with_bad, y_with_bad, verbose=True)
test_error_ls_bad = np.linalg.norm(X_hat @ w_ls_bad - y_test)
print("Testing error:", test_error_ls_bad)
print("\n")

# Ridge
ridge_lmbda = 10
w_rr_bad, train_error_rr_bad = ridge(X_hat_with_bad, y_with_bad, ridge_lmbda, verbose=True)
test_error_rr = np.linalg.norm(X_hat @ w_rr_bad - y_test)
print("Testing error:", test_error_rr)
print("\n")


# WLS: Only trust first half of data
W1_vec = np.concatenate((np.ones(X.shape[0]), np.zeros(X.shape[0])))
w_wls, train_error_wls = WLS(X_hat_with_bad, y_with_bad, W1_vec, verbose=True)
test_error_wls = np.linalg.norm(X_hat @ w_wls - y_test)
print("Testing error:", test_error_wls)
print("\n")

# Tikhonov:
# Constant weights on all terms
W2_vec = np.ones(X_hat.shape[1]) * 10

# Penalize distance to origin
w_0 = np.zeros(X_hat.shape[1])

w_tikh, train_error_tikh = tikhonov(X_hat_with_bad, y_with_bad, W1_vec, W2_vec, w_0, verbose=True)
test_error_tikh = np.linalg.norm(X_hat @ w_tikh - y_test)
print("Testing error:", test_error_tikh)
print("\n")

w_final = [[w_ls_bad, "OLS"], [w_rr_bad, "Ridge"], [w_wls, "WLS"], [w_tikh, "Tikhonov"]]

In [ ]:
plot_result(X_hat, X, X_true, y_obs, y_true, y_test, w_final, y_obs_bad=y_obs_bad)

## Weighting Polynomial Features (optional)
Using Tikhonov regularization, we have great control over how we want our regressor to end up. Don't like some of the data? Specify $W_1$ matrix to down weight such data. Interested in only allowing certain polynomial orders? Specify $W_2$ matrix to prefer such features. Below, you can see the result of designing the $W_2$ matrix to control the degree of our regressor.

In [ ]:
def degree_control(highest_degree_desired):
    # off-by-one due to existence of 0 degree terms
    allowed = np.ones(highest_degree_desired + 1)
    disallowed = np.ones(X_hat.shape[1] - (highest_degree_desired + 1)) * 10**15
    return np.concatenate((allowed, disallowed))

In [ ]:
for highest_degree in np.arange(X_hat.shape[1]):
    W2_vec = degree_control(highest_degree)
    w_tikh, train_error_tikh = tikhonov(X_hat_with_bad, y_with_bad, W1_vec, W2_vec, w_0, verbose=True)
    test_error_tikh = np.linalg.norm(X_hat @ w_tikh - y_test)
    print("Testing error:", test_error_tikh)
    print("\n")
    w_tikh_w2 = [[w_tikh, "Degree " + str(highest_degree)]]
    plot_result(X_hat, X, X_true, y_obs, y_true, y_test, w_tikh_w2, y_obs_bad=y_obs_bad)
    plt.show()

## Tikhonov Playground (optional)

Now that you have 3 hyperparameters, $W_1, W_2, \vec{w}_0$, there is so much fun to be had! Try out different values and see how it affects your regressor!

In [ ]:
# TODO: change these values to see what happens!

W1_vec = np.concatenate((np.ones(X.shape[0]), np.zeros(X.shape[0])))

W2_vec = np.ones(X_hat.shape[1]) * 10

w_0 = np.zeros(X_hat.shape[1])

In [ ]:
w_tikh, train_error_tikh = tikhonov(X_hat_with_bad, y_with_bad, W1_vec, W2_vec, w_0, verbose=True)
test_error_tikh = np.linalg.norm(X_hat @ w_tikh - y_test)
print("Testing error:", test_error_tikh)
print("\n")
w_tikh = [[w_tikh, "Tikhonov"]]
plot_result(X_hat, X, X_true, y_obs, y_true, y_test, w_tikh, y_obs_bad=y_obs_bad)

### TODO (optional): What hyperparmaters did you try? How did they affect the regressor?

# Extra Things to Try (optional)

Now that you've seen these methods in action, try and play around with different values. Some suggestions are to change data_fn() to generate a different set of data, or change the poly_degree argument in generate_data() to choose a different order polynomial regressor. It may be interesting to see what happens if you choose the data_fn() to be a lower order than the polynomial trying to fit it. There, overfitting will be on full display.